# S3 Access ICESat-2 ATL08 

In [1]:
# hide warning messages 
import warnings
warnings.filterwarnings('ignore')
# import python modules
import geopandas as gpd
import requests
import pandas as pd
import datetime as dt 
import h5py
import numpy as np
import s3fs
from os import path, remove
from posixpath import splitext
from requests.adapters import HTTPAdapter, Retry

The following input parameters are needed, passed through the front-end.

In [2]:
# start time
start_date = dt.datetime(2019, 1, 1) 
# end time
end_date = dt.datetime(2019, 1, 31)
# atl08 variables of interests as a list
variables = ['land_segments/canopy/h_canopy']
# path to polygon geojson file define subset area
poly_f = 'poly.json'

Let's define some additional variables.

In [3]:
# ICESat-2 reference orbits
orbits = gpd.read_file('orbits.json')
# nsidc s3 credentials
nsidc_s3 = f"https://data.nsidc.earthdatacloud.nasa.gov/s3credentials"
# cmr api url
cmrurl='https://cmr.earthdata.nasa.gov/search/' 
# ATL08 (Earthdata Cloud) concept_id
concept_id = 'C2153574670-NSIDC_CPRD' # 'C2144424132-NSIDC_ECS' is the public concept id
# CMR datetime format
dt_format = '%Y-%m-%dT%H:%M:%SZ'
# six beams of ICESat-2
beams = ['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']
# core variables to include in every subset request
headers = ['land_segments/latitude', 'land_segments/longitude', 'beam', 'granule']

Let's create a session to avoid 500 request errors.

In [4]:
# creating a backoff
s = requests.Session()
retries = Retry(total=3, backoff_factor=0.1, status_forcelist=[ 500, 502, 503, 504 ])
s.mount('https://', HTTPAdapter(max_retries=retries))

Let's find the intersecting ICESat-2 reference orbits intersecting with the polygon (`poly`).

In [5]:
poly = gpd.read_file(poly_f)
poly['geometry'] = poly.geometry.buffer(0.3, cap_style=3)
inter = gpd.sjoin(orbits, poly, how='inner', op='intersects')

The orbit column in the `inter` dataframe will have a four-digit reference ground track number of the intersecting orbits. We will now search CMR for ATL08 granules by passing this number to a CMR API request. This is necessary since the CMR bounding geometries of ATL08 are not representative of actual data bounds.

Accessing s3 links of atl08 via CMR requires authentication as it is not yet publicly available. Create a NASA EDL token first by going to `https://urs.earthdata.nasa.gov/users/<username>/user_tokens` and generate a token if you don't have one already. Save the token to the `~/.cmr_token` file in your home directory.

Now, let's define functions to search for ATL08 granules.

In [6]:
# function to read and cmr_token
def get_cmr_token(token_path: str):
    with open(token_path) as f:
        return f"Bearer {f.read().strip()}"

# function to search CMR 
def granule_search(concept_id: str, temporal: str, granule_id:str):
    page_num = 1
    granule_arr = []
    while True:
         # defining parameters
        cmr_param = {
            "collection_concept_id": concept_id, 
            "temporal": temporal,
            "page_size": 2000,
            "page_num": page_num,
            "options[producerGranuleId][pattern]": "true",
            "producerGranuleId": granule_id        }

        granulesearch = cmrurl + 'granules.json'
        token_path = path.join(path.expanduser("~"), '.cmr_token')
        response = requests.get(granulesearch, params=cmr_param, 
                                headers = {"Authorization" : get_cmr_token(token_path)})
        granules = response.json()['feed']['entry']
        if granules:
            for g in granules:
                # Get URL of HDF5 files
                for links in g['links']:
                    if links['href'].startswith('s3://') and links['href'].endswith('.h5'):
                        granule_arr.append(links['href'])

            page_num += 1
        else: 
            break
            
    return granule_arr

We now pass time bounds and four-digit ICESat-2 reference orbit numbers to the CMR search function defined above.

In [7]:
# CMR time bounds
temporal = start_date.strftime(dt_format) + ',' + end_date.strftime(dt_format)
alt08_files = []
for o_n in inter.orbit:
    alt08_files.extend(granule_search(concept_id, temporal, f"*_{str(o_n).zfill(4)}*"))

The list `alt08_files` will now have S3 links to ATL08 granules.

We will now get S3 temporary credentials and define the S3FS S3FileSystem object. Make sure that `.netrc` file is defined as described here: https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+cURL+And+Wget.

In [8]:
# get s3 credentials
s3credentials = s.get(nsidc_s3).json()
# defining S3FS object
fs_s3 = s3fs.S3FileSystem(anon=False, 
                          key=s3credentials['accessKeyId'], 
                          secret=s3credentials['secretAccessKey'], 
                          token=s3credentials['sessionToken'])

We will now define a function to spatially subset the ATL08 granules. 

In [9]:
def subset_atl08(t, var_l, poly, s3_list):
    outfiles = []
    for s3_url in sorted(s3_list):
        with fs_s3.open(s3_url, mode='rb') as fh:
            with h5py.File(fh) as hf:
                granule_f = path.basename(s3_url)
                out_csv = f'{splitext(granule_f)[0]}_{int(dt.datetime.utcnow().timestamp())}.csv'
                for var in list(hf.keys()):
                    if var.startswith('gt'):
                        if 'land_segments' in list(hf[var].keys()):
                            lat = hf[f'{var}/{var_l[0]}'][:]
                            lon = hf[f'{var}/{var_l[1]}'][:]
                            df = pd.DataFrame({var_l[0]: lat, var_l[1]: lon})
                            gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[var_l[1]], df[var_l[0]])) 
                            gdf_poly = gdf[gdf['geometry'].within(poly.geometry[0])]   
                            if not gdf_poly.empty:
                                if not path.exists(out_csv):
                                    outfiles.append(out_csv)
                                    with open(out_csv, "w") as f:
                                        f.write(','.join(var_l)+'\n')
                                        
                                gdf_poly['beam'] = np.repeat(str(var), len(gdf_poly.index)).tolist()
                                gdf_poly['granule'] = np.repeat(str(granule_f), len(gdf_poly.index)).tolist()

                                for v in var_l[t:]:
                                    gdf_poly[v] = None

                                # retrieving variables of interests
                                for _, df_gr in gdf_poly.groupby((gdf_poly.index.to_series().diff() > 1).cumsum()):
                                    i = df_gr.index.min()
                                    j = df_gr.index.max()
                                    for v in var_l[t:]:
                                        gdf_poly.loc[i:j, (v)] = hf[f'{var}/{v}'][i:j+1]

                                # saving the output file
                                gdf_poly.to_csv(out_csv, mode='a', index=False, header=False, columns=var_l)
    return outfiles

To the above function `subset_atl08`, we will pass a list `headers` and `variables` containing the variables of interest, a geopandas dataframe `poly_gpd` containing subset area polygon, and a list `alt08_files` containing the list of S3 links to ATL08 files.

In [10]:
# read the polygon json defining the subset area
poly_gpd = gpd.read_file(poly_f)
# subset
subset_f = subset_atl08(len(headers), headers+variables, poly_gpd, alt08_files)

The variable `subset_f` will now contain a list of CSV files that were created in the above step.  These CSV files contains the subset data. Now, we will read these subset CSV files and print out the results.

In [11]:
# reading the CSV
subset_df = pd.concat(pd.read_csv(f, header=0) for f in subset_f)
subset_df.reset_index(inplace=True)
# deleting the CSV
for f in subset_f:
    if path.isfile(f):
        remove(f)
# printing the pandas dataframe as json
subset_df.to_json()

'{"index":{"0":0,"1":1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9,"10":10,"11":11,"12":12,"13":13,"14":0,"15":1,"16":2,"17":3,"18":4,"19":5,"20":6,"21":7,"22":8,"23":9},"land_segments\\/latitude":{"0":-43.138596,"1":-43.13949,"2":-43.140385,"3":-43.141277,"4":-43.142174,"5":-43.143066,"6":-43.14396,"7":-43.144855,"8":-43.14575,"9":-43.146645,"10":-43.147537,"11":-43.140118,"12":-43.1428,"13":-43.147274,"14":-43.147022,"15":-43.146126,"16":-43.145233,"17":-43.14434,"18":-43.143444,"19":-43.14255,"20":-43.141655,"21":-43.14076,"22":-43.139866,"23":-43.13897},"land_segments\\/longitude":{"0":147.36943,"1":147.36931,"2":147.36919,"3":147.36906,"4":147.36894,"5":147.36882,"6":147.36871,"7":147.36859,"8":147.36847,"9":147.36835,"10":147.36823,"11":147.36794,"12":147.36758,"13":147.367,"14":147.3681,"15":147.36798,"16":147.36786,"17":147.36774,"18":147.36761,"19":147.36751,"20":147.36739,"21":147.36728,"22":147.36716,"23":147.36703},"beam":{"0":"gt1l","1":"gt1l","2":"gt1l","3":"gt1l","4"